Ref: https://stackabuse.com/python-for-nlp-neural-machine-translation-with-seq2seq-in-keras/

In [1]:
# opening the train_source file in read mode
my_file = open("train-source.txt", "r", encoding='UTF-8')

# reading the file
data = my_file.read()

train_source_list = data.replace('\n'," ").split('</s>')

my_file.close()

In [2]:
# opening the train-target file in read mode
my_file = open("train-target.txt", "r", encoding='UTF-8')

# reading the file
data = my_file.read()

train_target_list = data.replace('\n'," ").split('</s>')

my_file.close()

In [ ]:
# opening the test-target file in read mode
my_file = open("test-source.txt", "r", encoding='UTF-8')

# reading the file
data = my_file.read()

test_source_list = data.replace('\n'," ").split('</s>')

my_file.close()

# opening the test-target file in read mode
my_file = open("test-target.txt", "r", encoding='UTF-8')

# reading the file
data = my_file.read()

test_target_list = data.replace('\n'," ").split('</s>')

my_file.close()

In [ ]:
print("length of training sets Source: ", len(train_source_list), " Target :", len(train_target_list))
print("length of test sets Source: ", len(test_source_list), " Target :", len(test_target_list))

In [4]:
import os, sys

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [7]:
input_sentences = train_source_list
output_sentences = train_target_list
output_sentences_inputs = output_sentences


print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))
print("num samples output input:", len(output_sentences_inputs))

num samples input: 45172
num samples output: 45172
num samples output input: 45172


In [8]:
print(input_sentences[172])
print(output_sentences[172])
print(output_sentences_inputs[172])

 <s> Ní rabh aon duine fá'n áit a rabh fhios aige dadadh fá dtaobh de . 
 <s> Ní raibh aon duine faoin áit a raibh a fhios aige dada fá dtaobh de . 
 <s> Ní raibh aon duine faoin áit a raibh a fhios aige dada fá dtaobh de . 


In [10]:
input_unique_words = set()
for sentence in train_source_list:
    input_unique_words.update(sentence)

In [12]:
print("no. of unique word in input: ",len(input_unique_words))
MAX_NUM_WORDS=len(input_unique_words)

no. of unique word in input:  107


In [13]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 27496
Length of longest sentence in input: 104


In [ ]:
word2idx_inputs

In [14]:
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 25738
Length of longest sentence in the output: 103


In [18]:
BATCH_SIZE = 100
EPOCHS = 2
LSTM_NODES =232
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = max_out_len
MAX_NUM_WORDS = len(input_unique_words)
EMBEDDING_SIZE = 100

In [15]:
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", encoder_input_sequences.shape)
print("encoder_input_sequences[172]:", encoder_input_sequences[172])

encoder_input_sequences.shape: (45172, 104)
encoder_input_sequences[172]: [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   1  14  19  90  62  89   2  19 104  61  52  29]


In [16]:
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[172]:", decoder_input_sequences[172])

decoder_input_sequences.shape: (45172, 103)
decoder_input_sequences[172]: [  1  19  18  64  56 100   3  18   3 104  60  75  29   2   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [22]:
input_token_index = dict([(char, i) for i, char in enumerate(train_source_list)])
target_token_index = dict([(char, i) for i, char in enumerate(train_target_list)])

print(len(input_token_index))
print(len(target_token_index))

44981
44930


In [40]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = input_token_index

# for line in glove_file:
#     records = line.split()
#     word = records[0]
#     vector_dimensions = asarray(records[1:], dtype='float64')
#     embeddings_dictionary[word] = vector_dimensions
# glove_file.close()

In [41]:
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = zeros((num_words, EMBEDDING_SIZE))
for word, index in word2idx_inputs.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [42]:
embeddings_dictionary['agus']

KeyError: 'agus'

In [45]:
print(embedding_matrix['agus'])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [46]:
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)

In [47]:
decoder_targets_one_hot = np.zeros((
        len(input_sentences),
        max_out_len,
        num_words_output
    ),
    dtype='float32'
)

MemoryError: Unable to allocate 446. GiB for an array with shape (45172, 103, 25739) and data type float32